# 语音控制堆叠

In [ ]:
#导入相关的模块
import threading
from Arm_Lib import Arm_Device
import time

Arm = Arm_Device()

# 定义机械臂控制函数

In [ ]:
# 定义移动机械臂函数,同时控制1-6号舵机运动，p=[S1,S2,S3,S4,S5,S6]
def arm_move_6(p, s_time = 500):
    for i in range(6):
        id = i + 1
        Arm.Arm_serial_servo_write(id, p[i], s_time)
        time.sleep(.01)
    time.sleep(s_time/1000)
    
# 定义移动机械臂函数,同时控制1-5号舵机运动，p=[S1,S2,S3,S4,S5]
def arm_move(p, s_time = 500):
    for i in range(5):
        id = i + 1
        if id == 5:
            time.sleep(.1)
            Arm.Arm_serial_servo_write(id, p[i], int(s_time*1.2))
        elif id == 1 :
            Arm.Arm_serial_servo_write(id, p[i], int(3*s_time/4))
        else:
            Arm.Arm_serial_servo_write(id, p[i], int(s_time))
        time.sleep(.01)
    time.sleep(s_time/1000)
    
# 定义夹积木块函数，enable=1：夹住，=0：松开
def arm_clamp_block(enable):
    if enable == 0:
        Arm.Arm_serial_servo_write(6, 60, 400)
    else:
        Arm.Arm_serial_servo_write(6, 135, 400)
    time.sleep(.5)

In [ ]:
# 定义不同位置的变量参数
look_at = [90, 164, 18, 0, 90, 90]
p_top = [90, 80, 50, 50, 270]

p_Yellow = [65, 22, 64, 56, 270]
p_Red = [118, 19, 66, 56, 270]

p_Green = [136, 66, 20, 29, 270]
p_Blue = [44, 66, 20, 28, 270]

p_layer_4 = [90, 76, 40, 17, 270]
p_layer_3 = [90, 65, 44, 17, 270]
p_layer_2 = [90, 65, 25, 36, 270]
p_layer_1 = [90, 48, 35, 30, 270]


p_push_over_1 = [90, 90, 5, 0, 90, 150]
p_push_over_2 = [90, 90, 0, 50, 90, 150]


#定义抓取方块的状态
yellow_grabbed = 0
red_grabbed = 0
green_grabbed = 0
blue_grabbed = 0


#定义手势识别次数
Count_One = 0
Count_Two = 0
Count_Three = 0
Count_Four = 0
Count_Fist = 0


In [ ]:
arm_move_6(look_at, 1000)
time.sleep(1)

In [ ]:
#数字功能定义
def number_action(index):
    if index == 1:
        # 抓取黄色的积木块
        arm_move(p_top, 1000)
        arm_move(p_Yellow, 1000)
        arm_clamp_block(1)
#         time.sleep(.5)
        arm_move(p_top, 1000)
    elif index == 2:
        # 抓取红色的积木块
        arm_move(p_top, 1000)
        arm_move(p_Red, 1000)
        arm_clamp_block(1)
        arm_move(p_top, 1000)
    elif index == 3:
        # 抓取绿色的积木块
        arm_move(p_top, 1000)
        arm_move(p_Green, 1000)
        arm_clamp_block(1)
        arm_move(p_top, 1000)
    elif index == 4:
        # 抓取蓝色的积木块
        arm_move(p_top, 1000)
        arm_move(p_Blue, 1000)
        arm_clamp_block(1)
        arm_move(p_top, 1000)

    
def put_down_block(layer):
    
    if layer == 1:
        arm_move(p_layer_1, 1000)
        arm_clamp_block(0) 
        arm_move_6(look_at, 1000)
    elif layer == 2:
        arm_move(p_layer_2, 1000)
        arm_clamp_block(0)
        arm_move_6(look_at, 1000)
    elif layer == 3:
        arm_move(p_layer_3, 1000)
        arm_clamp_block(0) 
        arm_move_6(look_at, 1000)
    elif layer == 4:
        arm_move(p_layer_4, 1000)
        time.sleep(.1)
        arm_clamp_block(0) 
        arm_move_6(look_at, 1000)
    Arm.Arm_ask_speech(32)
    time.sleep(0.1)   
    
# 推倒积木块
def push_over_block():
    arm_move_6(p_push_over_1, 1000)
    time.sleep(.2)
    arm_move_6(p_push_over_2, 1000)
    time.sleep(.1)
    arm_move_6(look_at, 1000)
    time.sleep(1)
    global g_layer
    g_layer = 0


In [ ]:
global g_state_arm
g_state_arm = 0

global g_layer
g_layer = 0

def ctrl_arm_move(index):
    global g_layer
    g_layer = g_layer + 1
    if g_layer >= 5:
        g_layer = 1
    arm_clamp_block(0)
    if index == 1:
        number_action(index)
        time.sleep(1)
        put_down_block(g_layer)
    elif index == 2:
        number_action(index)
        time.sleep(1)
        put_down_block(g_layer)
    elif index == 3:
        number_action(index)
        time.sleep(1)
        put_down_block(g_layer)
    elif index == 4:
        number_action(index)
        time.sleep(1)
        put_down_block(g_layer)
    elif index == 5:
        time.sleep(1)
        push_over_block()
    
        
    global g_state_arm
    g_state_arm = 0


In [ ]:
def start_move_arm(index):
    # 开启机械臂控制线程
    global g_state_arm
    if g_state_arm == 0:
        closeTid = threading.Thread(target = ctrl_arm_move, args = [index])
        closeTid.setDaemon(True)
        closeTid.start()
        
        g_state_arm = 1

In [ ]:
Arm.Arm_ask_speech(9)
time.sleep(0.001)

# 主进程

In [ ]:
 try:
    Arm.Arm_Buzzer_On(1)
    s_time = 300
    Arm.Arm_serial_servo_write(4, 10, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 0, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 10, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 0, s_time)
    time.sleep(s_time/1000)
    while True:
        res = Arm.Arm_serial_speech_read(1)
  
        if res == 28:
            if yellow_grabbed == 0:
                # Speech_text("好的，开始夹取",EncodingFormat_Type["GB2312"])
                # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                #     time.sleep(0.1)  
                Arm.Arm_Buzzer_On(1)
                time.sleep(.1)
                start_move_arm(1)
                time.sleep(1)
                # global yellow_grabbed
                yellow_grabbed = 1
        elif res == 29:
            if red_grabbed == 0:
                # Speech_text("好的，开始夹取",EncodingFormat_Type["GB2312"])
                # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                #     time.sleep(0.1)  
                Arm.Arm_Buzzer_On(1)
                time.sleep(.1)
                start_move_arm(2)
                # global red_grabbed
                red_grabbed = 1
        elif res == 30:
            if green_grabbed == 0:
                # Speech_text("好的，开始夹取",EncodingFormat_Type["GB2312"])
                # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                #     time.sleep(0.1)  
                Arm.Arm_Buzzer_On(1)
                time.sleep(.1)
                start_move_arm(3)
                # global green_grabbed
                green_grabbed = 1
        elif res == 31:
            if blue_grabbed == 0:
                # Speech_text("好的，开始夹取",EncodingFormat_Type["GB2312"])
                # while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:#等待当前语句播报结束
                #     time.sleep(0.1)  
                Arm.Arm_Buzzer_On(1)
                time.sleep(.1)
                start_move_arm(4)
                # global blue_grabbed
                blue_grabbed = 1
        elif res == 32: #拳头
            Arm.Arm_Buzzer_On(1)
            time.sleep(.1)
            start_move_arm(5)
            yellow_grabbed = 0
            red_grabbed = 0
            green_grabbed = 0
            blue_grabbed = 0
            Count_Fist = 0
except KeyboardInterrupt:
    print(" Program closed! ")
    pass
